In [1]:
import cv2
print('opencv: ' +  cv2.__version__)

import numpy as np
import math

from Odometry2D import VisualOdometry
from PinholeCamera import PinholeCamera
from OdometryUtil import truncate

opencv: 3.4.3


### Camera settings

In [2]:
cam = PinholeCamera(1241.0, 376.0, 718.8560, 718.8560, 607.1928, 185.2157)

### Output settings

In [3]:
OUT_W = 1000
OUT_H = 1100

START_X = 500
START_Y = 550

RES_W = 1241
RES_H = 376

traj = np.zeros((OUT_W, OUT_H, 3), dtype=np.uint8)

font = cv2.FONT_HERSHEY_TRIPLEX
fontColor = (255, 255, 0)
trajColor = (0, 255, 255)

### Visual Odometry

In [4]:
vo = VisualOdometry(cam, True)
vo.readAnnotations('D:/Data/dataset/poses/00.txt')

for img_id in range(4541):
    time1 = cv2.getTickCount()
    img = cv2.imread('D:/Data/data_odometry_gray/dataset/sequences/00/image_0/' + str(img_id).zfill(6) + '.png', 0)

    vo.update(img, img_id)
    
    cur_t = vo.cur_t
    if(img_id > 2):
        x, y, z = cur_t[0], cur_t[1], cur_t[2]
    else:
        x, y, z = 0., 0., 0.
    draw_x, draw_y = int(x)+START_X, int(z)+START_Y
    true_x, true_y = int(vo.trueX)+START_X, int(vo.trueZ)+START_Y

    cv2.circle(traj, (draw_x,draw_y), 1, (img_id*255/4540,255-img_id*255/4540,0), 1)
    cv2.circle(traj, (true_x,true_y), 1, (0,0,255), 2)
    cv2.rectangle(traj, (10, 20), (600, 60), (0,0,0), -1)
    text = "Coordinates: x=%2fm y=%2fm z=%2fm"%(x,y,z)
    cv2.putText(traj, text, (20,40), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, 8)
    
    time2 = cv2.getTickCount()
    time = math.floor(((time2 - time1) * 1000) / cv2.getTickFrequency())
    imgOut = cv2.copyMakeBorder(img, 0, 40, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    textString = str(time) + " ms "
    cv2.putText(imgOut, textString, (10, RES_H + 40), font, 1, fontColor, 2, 8)

    cv2.imshow('Road facing camera', imgOut)
    cv2.imshow('Trajectory', traj)
    cv2.waitKey(1)

cv2.imwrite('map.png', traj)

KeyboardInterrupt: 